### aim: 

### date: 

In [30]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [31]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [32]:
import numpy as np
from cmocean import cm
import cartopy as cp
import cartopy.crs as ccrs
import netCDF4 as nc
import matplotlib.pyplot as plt
import xarray as xr
import sys
sys.path.append('/gpfs/home/mep22dku/scratch/SOZONE')
#list of models
sys.path.append('/gpfs/home/mep22dku/scratch/SOZONE/UTILS')
import lom
import utils as ut

%matplotlib inline
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')
import cartopy.feature as cfeature
from importlib import reload
import matplotlib.path as mpath
import glob
import pickle
import pandas as pd
import seawater
import time
plt.rcParams.update({'font.size': 12})
font = {'family' : 'normal',
'weight' : 'normal',
'size'   : 12}

plt.rc('font', **font)

In [33]:
import pymannkendall as mk
from scipy import stats

def give_trends(ts_x,ts_y):

    trend, h, mk_p, z, Tau, s, var_s, mk_slope, intercept = mk.original_test(ts_y)
    lin_slope, intercept, r_value, lin_p, std_err = stats.linregress(ts_x,ts_y)
    
    return mk_slope, mk_p, lin_slope, lin_p

In [34]:
def make_yearlist_UKESM(yrst, yrend, scen = '1A'):
    
    baseDir = '//gpfs/data/greenocean/software/products/windsFromComponents/symlinked_hrly_UKESM/daily/'

    yrs = np.arange(yrst,yrend+1,1)
    ylist = []
    for i in range(0,len(yrs)):
        ty = f'{baseDir}/UKESM_{scen}_y{yrs[i]}_wspd10m_seas_rg.nc'
        t2 = glob.glob(ty)
        #print(t2)
        ylist.append(t2[0])
    return ylist


ylist_1A = xr.open_mfdataset(make_yearlist_UKESM(2020,2099,'1A'))
ylist_1B = xr.open_mfdataset(make_yearlist_UKESM(2020,2099,'1B'))
ylist_2A = xr.open_mfdataset(make_yearlist_UKESM(2020,2099,'2A'))
ylist_2B = xr.open_mfdataset(make_yearlist_UKESM(2020,2099,'2B'))
ylist_3A = xr.open_mfdataset(make_yearlist_UKESM(2020,2099,'3A'))
ylist_3B = xr.open_mfdataset(make_yearlist_UKESM(2020,2099,'3B'))                                              

In [35]:
ylist_year = ylist_1A.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_1A['time_counter.season'] == 'DJF')).\
groupby('time_counter.year').mean().year.values

In [36]:
ylist_year

array([2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030,
       2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041,
       2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052,
       2053, 2054, 2055, 2056, 2057, 2058, 2059, 2060, 2061, 2062, 2063,
       2064, 2065, 2066, 2067, 2068, 2069, 2070, 2071, 2072, 2073, 2074,
       2075, 2076, 2077, 2078, 2079, 2080, 2081, 2082, 2083, 2084, 2085,
       2086, 2087, 2088, 2089, 2090, 2091, 2092, 2093, 2094, 2095, 2096,
       2097, 2098, 2099])

In [37]:
horse = True
if horse:
    ylist_year = ylist_1A.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_1A['time_counter.season'] == 'DJF')).\
    groupby('time_counter.year').mean().year.values

    ylist_1A_DJF = ylist_1A.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_1A['time_counter.season'] == 'DJF')).\
    groupby('time_counter.year').mean().values
    ylist_1A_MAM = ylist_1A.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_1A['time_counter.season'] == 'MAM')).\
    groupby('time_counter.year').mean().values
    ylist_1A_JJA = ylist_1A.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_1A['time_counter.season'] == 'JJA')).\
    groupby('time_counter.year').mean().values
    ylist_1A_SON = ylist_1A.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_1A['time_counter.season'] == 'SON')).\
    groupby('time_counter.year').mean().values
    ylist_1A_FY = ylist_1A.wspd10m.isel(lat=slice(0,60)).\
    groupby('time_counter.year').mean().values
    np.shape(ylist_1A_FY)
    
    w = time.time()

    tdats = [ylist_1A_FY, ylist_1A_DJF, ylist_1A_MAM,ylist_1A_JJA, ylist_1A_SON]

    mk_slope = np.zeros([5,60,360])
    mk_p = np.zeros([5,60,360])

    for sc in range(0,5):
        testdat = tdats[sc]
        print(sc)
        for i in range(0,60):
            for j in range(0,360):
                tda = testdat[:,i,j]
                lin_slope, intercept, r_value, lin_p, std_err = stats.linregress(ylist_year,tda)
                mk_slope[sc,i,j] = lin_slope
                mk_p[sc,i,j] = lin_p

    savenam = '/gpfs/data/greenocean/software/products/windsFromComponents/MannKendall/UKESM_1A_lin-2020-2099.nc'
    data_vars = {'lin_slope':(['season', 'lat', 'lon'], mk_slope,
    {'long_name':'lin reg, 1940-2020'}),
                 'lin_p':(['season', 'lat', 'lon'], mk_p,
    {'long_name':'lin reg, 1940-2020'}),
    }

    lats = ylist_1A.lat.isel(lat = slice(0,60)).values
    lons = ylist_1A.lon.values
    seasons = ['FY','DJF','MAM','JJA','SON']
    # define coordinates
    coords = {'season': (['season'], seasons),
            'lat': (['lat'], lats),
    'lon': (['lon'], lons),
             }
    # define global attributes
    attrs = {'made in':'SOZONE/windAnalyis/wspdComponents/Fig-futmap-winds.ipynb',
    'desc': ''
    }
    ds = xr.Dataset(data_vars=data_vars,
    coords=coords,
    attrs=attrs)
    ds.to_netcdf(savenam)

    w2 = time.time()
    print(w2-w)

    ylist_year = ylist_1B.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_1B['time_counter.season'] == 'DJF')).\
    groupby('time_counter.year').mean().year.values

    ylist_1B_DJF = ylist_1B.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_1B['time_counter.season'] == 'DJF')).\
    groupby('time_counter.year').mean().values
    ylist_1B_MAM = ylist_1B.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_1B['time_counter.season'] == 'MAM')).\
    groupby('time_counter.year').mean().values
    ylist_1B_JJA = ylist_1B.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_1B['time_counter.season'] == 'JJA')).\
    groupby('time_counter.year').mean().values
    ylist_1B_SON = ylist_1B.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_1B['time_counter.season'] == 'SON')).\
    groupby('time_counter.year').mean().values
    ylist_1B_FY = ylist_1B.wspd10m.isel(lat=slice(0,60)).\
    groupby('time_counter.year').mean().values
    np.shape(ylist_1B_FY)
    
    w = time.time()

    tdats = [ylist_1B_FY, ylist_1B_DJF, ylist_1B_MAM,ylist_1B_JJA, ylist_1B_SON]

    mk_slope = np.zeros([5,60,360])
    mk_p = np.zeros([5,60,360])

    for sc in range(0,5):
        testdat = tdats[sc]
        print(sc)
        for i in range(0,60):
            for j in range(0,360):
                tda = testdat[:,i,j]
                lin_slope, intercept, r_value, lin_p, std_err = stats.linregress(ylist_year,tda)
                mk_slope[sc,i,j] = lin_slope
                mk_p[sc,i,j] = lin_p

    savenam = '/gpfs/data/greenocean/software/products/windsFromComponents/MannKendall/UKESM_1B_lin-2020-2099.nc'
    data_vars = {'lin_slope':(['season', 'lat', 'lon'], mk_slope,
    {'long_name':'lin reg, 1940-2020'}),
                 'lin_p':(['season', 'lat', 'lon'], mk_p,
    {'long_name':'lin reg, 1940-2020'}),
    }

    lats = ylist_1B.lat.isel(lat = slice(0,60)).values
    lons = ylist_1B.lon.values
    seasons = ['FY','DJF','MAM','JJA','SON']
    # define coordinates
    coords = {'season': (['season'], seasons),
            'lat': (['lat'], lats),
    'lon': (['lon'], lons),
             }
    # define global attributes
    attrs = {'made in':'SOZONE/windAnalyis/wspdComponents/Fig-futmap-winds.ipynb',
    'desc': ''
    }
    ds = xr.Dataset(data_vars=data_vars,
    coords=coords,
    attrs=attrs)
    ds.to_netcdf(savenam)

    w2 = time.time()
    print(w2-w)



0
1
2
3
4
17.57597541809082
0
1
2
3
4
17.536275148391724


In [ ]:
horse = True
if horse:
    ylist_year = ylist_2A.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_2A['time_counter.season'] == 'DJF')).\
    groupby('time_counter.year').mean().year.values

    ylist_2A_DJF = ylist_2A.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_2A['time_counter.season'] == 'DJF')).\
    groupby('time_counter.year').mean().values
    ylist_2A_MAM = ylist_2A.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_2A['time_counter.season'] == 'MAM')).\
    groupby('time_counter.year').mean().values
    ylist_2A_JJA = ylist_2A.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_2A['time_counter.season'] == 'JJA')).\
    groupby('time_counter.year').mean().values
    ylist_2A_SON = ylist_2A.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_2A['time_counter.season'] == 'SON')).\
    groupby('time_counter.year').mean().values
    ylist_2A_FY = ylist_2A.wspd10m.isel(lat=slice(0,60)).\
    groupby('time_counter.year').mean().values
    np.shape(ylist_2A_FY)
    
    w = time.time()

    tdats = [ylist_2A_FY, ylist_2A_DJF, ylist_2A_MAM,ylist_2A_JJA, ylist_2A_SON]

    mk_slope = np.zeros([5,60,360])
    mk_p = np.zeros([5,60,360])

    for sc in range(0,5):
        testdat = tdats[sc]
        print(sc)
        for i in range(0,60):
            for j in range(0,360):
                tda = testdat[:,i,j]
                lin_slope, intercept, r_value, lin_p, std_err = stats.linregress(ylist_year,tda)
                mk_slope[sc,i,j] = lin_slope
                mk_p[sc,i,j] = lin_p

    savenam = '/gpfs/data/greenocean/software/products/windsFromComponents/MannKendall/UKESM_2A_lin-2020-2099.nc'
    data_vars = {'lin_slope':(['season', 'lat', 'lon'], mk_slope,
    {'long_name':'lin reg, 1940-2020'}),
                 'lin_p':(['season', 'lat', 'lon'], mk_p,
    {'long_name':'lin reg, 1940-2020'}),
    }

    lats = ylist_2A.lat.isel(lat = slice(0,60)).values
    lons = ylist_2A.lon.values
    seasons = ['FY','DJF','MAM','JJA','SON']
    # define coordinates
    coords = {'season': (['season'], seasons),
            'lat': (['lat'], lats),
    'lon': (['lon'], lons),
             }
    # define global attributes
    attrs = {'made in':'SOZONE/windAnalyis/wspdComponents/Fig-futmap-winds.ipynb',
    'desc': ''
    }
    ds = xr.Dataset(data_vars=data_vars,
    coords=coords,
    attrs=attrs)
    ds.to_netcdf(savenam)

    w2 = time.time()
    print(w2-w)

    ylist_year = ylist_2B.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_2B['time_counter.season'] == 'DJF')).\
    groupby('time_counter.year').mean().year.values

    ylist_2B_DJF = ylist_2B.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_2B['time_counter.season'] == 'DJF')).\
    groupby('time_counter.year').mean().values
    ylist_2B_MAM = ylist_2B.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_2B['time_counter.season'] == 'MAM')).\
    groupby('time_counter.year').mean().values
    ylist_2B_JJA = ylist_2B.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_2B['time_counter.season'] == 'JJA')).\
    groupby('time_counter.year').mean().values
    ylist_2B_SON = ylist_2B.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_2B['time_counter.season'] == 'SON')).\
    groupby('time_counter.year').mean().values
    ylist_2B_FY = ylist_2B.wspd10m.isel(lat=slice(0,60)).\
    groupby('time_counter.year').mean().values
    np.shape(ylist_2B_FY)
    
    w = time.time()

    tdats = [ylist_2B_FY, ylist_2B_DJF, ylist_2B_MAM,ylist_2B_JJA, ylist_2B_SON]

    mk_slope = np.zeros([5,60,360])
    mk_p = np.zeros([5,60,360])

    for sc in range(0,5):
        testdat = tdats[sc]
        print(sc)
        for i in range(0,60):
            for j in range(0,360):
                tda = testdat[:,i,j]
                lin_slope, intercept, r_value, lin_p, std_err = stats.linregress(ylist_year,tda)
                mk_slope[sc,i,j] = lin_slope
                mk_p[sc,i,j] = lin_p

    savenam = '/gpfs/data/greenocean/software/products/windsFromComponents/MannKendall/UKESM_2B_lin-2020-2099.nc'
    data_vars = {'lin_slope':(['season', 'lat', 'lon'], mk_slope,
    {'long_name':'lin reg, 1940-2020'}),
                 'lin_p':(['season', 'lat', 'lon'], mk_p,
    {'long_name':'lin reg, 1940-2020'}),
    }

    lats = ylist_2B.lat.isel(lat = slice(0,60)).values
    lons = ylist_2B.lon.values
    seasons = ['FY','DJF','MAM','JJA','SON']
    # define coordinates
    coords = {'season': (['season'], seasons),
            'lat': (['lat'], lats),
    'lon': (['lon'], lons),
             }
    # define global attributes
    attrs = {'made in':'SOZONE/windAnalyis/wspdComponents/Fig-futmap-winds.ipynb',
    'desc': ''
    }
    ds = xr.Dataset(data_vars=data_vars,
    coords=coords,
    attrs=attrs)
    ds.to_netcdf(savenam)

    w2 = time.time()
    print(w2-w)



0
1
2
3
4
17.676501274108887
0
1
2
3


In [ ]:
horse = True
if horse:
    ylist_year = ylist_3A.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_3A['time_counter.season'] == 'DJF')).\
    groupby('time_counter.year').mean().year.values

    ylist_3A_DJF = ylist_3A.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_3A['time_counter.season'] == 'DJF')).\
    groupby('time_counter.year').mean().values
    ylist_3A_MAM = ylist_3A.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_3A['time_counter.season'] == 'MAM')).\
    groupby('time_counter.year').mean().values
    ylist_3A_JJA = ylist_3A.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_3A['time_counter.season'] == 'JJA')).\
    groupby('time_counter.year').mean().values
    ylist_3A_SON = ylist_3A.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_3A['time_counter.season'] == 'SON')).\
    groupby('time_counter.year').mean().values
    ylist_3A_FY = ylist_3A.wspd10m.isel(lat=slice(0,60)).\
    groupby('time_counter.year').mean().values
    np.shape(ylist_3A_FY)
    
    w = time.time()

    tdats = [ylist_3A_FY, ylist_3A_DJF, ylist_3A_MAM,ylist_3A_JJA, ylist_3A_SON]

    mk_slope = np.zeros([5,60,360])
    mk_p = np.zeros([5,60,360])

    for sc in range(0,5):
        testdat = tdats[sc]
        print(sc)
        for i in range(0,60):
            for j in range(0,360):
                tda = testdat[:,i,j]
                lin_slope, intercept, r_value, lin_p, std_err = stats.linregress(ylist_year,tda)
                mk_slope[sc,i,j] = lin_slope
                mk_p[sc,i,j] = lin_p

    savenam = '/gpfs/data/greenocean/software/products/windsFromComponents/MannKendall/UKESM_3A_lin-2020-2099.nc'
    data_vars = {'lin_slope':(['season', 'lat', 'lon'], mk_slope,
    {'long_name':'lin reg, 1940-2020'}),
                 'lin_p':(['season', 'lat', 'lon'], mk_p,
    {'long_name':'lin reg, 1940-2020'}),
    }

    lats = ylist_3A.lat.isel(lat = slice(0,60)).values
    lons = ylist_3A.lon.values
    seasons = ['FY','DJF','MAM','JJA','SON']
    # define coordinates
    coords = {'season': (['season'], seasons),
            'lat': (['lat'], lats),
    'lon': (['lon'], lons),
             }
    # define global attributes
    attrs = {'made in':'SOZONE/windAnalyis/wspdComponents/Fig-futmap-winds.ipynb',
    'desc': ''
    }
    ds = xr.Dataset(data_vars=data_vars,
    coords=coords,
    attrs=attrs)
    ds.to_netcdf(savenam)

    w2 = time.time()
    print(w2-w)

    ylist_year = ylist_3B.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_3B['time_counter.season'] == 'DJF')).\
    groupby('time_counter.year').mean().year.values

    ylist_3B_DJF = ylist_3B.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_3B['time_counter.season'] == 'DJF')).\
    groupby('time_counter.year').mean().values
    ylist_3B_MAM = ylist_3B.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_3B['time_counter.season'] == 'MAM')).\
    groupby('time_counter.year').mean().values
    ylist_3B_JJA = ylist_3B.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_3B['time_counter.season'] == 'JJA')).\
    groupby('time_counter.year').mean().values
    ylist_3B_SON = ylist_3B.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_3B['time_counter.season'] == 'SON')).\
    groupby('time_counter.year').mean().values
    ylist_3B_FY = ylist_3B.wspd10m.isel(lat=slice(0,60)).\
    groupby('time_counter.year').mean().values
    np.shape(ylist_3B_FY)
    
    w = time.time()

    tdats = [ylist_3B_FY, ylist_3B_DJF, ylist_3B_MAM,ylist_3B_JJA, ylist_3B_SON]

    mk_slope = np.zeros([5,60,360])
    mk_p = np.zeros([5,60,360])

    for sc in range(0,5):
        testdat = tdats[sc]
        print(sc)
        for i in range(0,60):
            for j in range(0,360):
                tda = testdat[:,i,j]
                lin_slope, intercept, r_value, lin_p, std_err = stats.linregress(ylist_year,tda)
                mk_slope[sc,i,j] = lin_slope
                mk_p[sc,i,j] = lin_p

    savenam = '/gpfs/data/greenocean/software/products/windsFromComponents/MannKendall/UKESM_3B_lin-2020-2099.nc'
    data_vars = {'lin_slope':(['season', 'lat', 'lon'], mk_slope,
    {'long_name':'lin reg, 1940-2020'}),
                 'lin_p':(['season', 'lat', 'lon'], mk_p,
    {'long_name':'lin reg, 1940-2020'}),
    }

    lats = ylist_3B.lat.isel(lat = slice(0,60)).values
    lons = ylist_3B.lon.values
    seasons = ['FY','DJF','MAM','JJA','SON']
    # define coordinates
    coords = {'season': (['season'], seasons),
            'lat': (['lat'], lats),
    'lon': (['lon'], lons),
             }
    # define global attributes
    attrs = {'made in':'SOZONE/windAnalyis/wspdComponents/Fig-futmap-winds.ipynb',
    'desc': ''
    }
    ds = xr.Dataset(data_vars=data_vars,
    coords=coords,
    attrs=attrs)
    ds.to_netcdf(savenam)

    w2 = time.time()
    print(w2-w)



In [ ]:
plt.pcolormesh(ds.lin_slope[1,:,:], vmin = -0.05, vmax = 0.05, cmap = cm.balance)

In [ ]:
horse = False
if horse:
    
    UKESM_DJF = ylist_UKESM.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_ERA['time_counter.season'] == 'DJF')).\
    groupby('time_counter.year').mean().values
    UKESM_MAM = ylist_UKESM.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_ERA['time_counter.season'] == 'MAM')).\
    groupby('time_counter.year').mean().values
    UKESM_JJA = ylist_UKESM.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_ERA['time_counter.season'] == 'JJA')).\
    groupby('time_counter.year').mean().values
    UKESM_SON = ylist_UKESM.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_ERA['time_counter.season'] == 'SON')).\
    groupby('time_counter.year').mean().values
    UKESM_FY = ylist_UKESM.wspd10m.isel(lat=slice(0,60)).\
    groupby('time_counter.year').mean().values
    np.shape(UKESM_FY)

    ERA_DJF = ylist_ERA.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_ERA['time_counter.season'] == 'DJF')).\
    groupby('time_counter.year').mean().values
    ERA_MAM = ylist_ERA.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_ERA['time_counter.season'] == 'MAM')).\
    groupby('time_counter.year').mean().values
    ERA_JJA = ylist_ERA.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_ERA['time_counter.season'] == 'JJA')).\
    groupby('time_counter.year').mean().values
    ERA_SON = ylist_ERA.wspd10m.isel(lat=slice(0,60)).sel(time_counter=(ylist_ERA['time_counter.season'] == 'SON')).\
    groupby('time_counter.year').mean().values
    ERA_FY = ylist_ERA.wspd10m.isel(lat=slice(0,60)).\
    groupby('time_counter.year').mean().values
    np.shape(ERA_FY)
    
    w = time.time()

    tdats = [UKESM_FY, UKESM_DJF, UKESM_MAM,UKESM_JJA, UKESM_SON]

    mk_slope = np.zeros([5,60,360])
    mk_p = np.zeros([5,60,360])

    for sc in range(0,5):
        testdat = tdats[sc]
        print(sc)
        for i in range(0,60):
            print(i)
            for j in range(0,360):
                tda = testdat[:,i,j]
                trend, h, p, z, Tau, s, var_s, slope, intercept = mk.original_test(tda)
                mk_slope[sc,i,j] = slope
                mk_p[sc,i,j] = p

    savenam = '/gpfs/data/greenocean/software/products/windsFromComponents/MannKendall/UKESM_1A_MK-1940-2020.nc'
    data_vars = {'mk_slope':(['season', 'lat', 'lon'], mk_slope,
    {'long_name':'MK, 1940-2020'}),
                 'mk_p':(['season', 'lat', 'lon'], mk_p,
    {'long_name':'MK, 1940-2020'}),
    }

    lats = ylist_UKESM.lat.isel(lat = slice(0,60)).values
    lons = ylist_UKESM.lon.values
    seasons = ['FY','DJF','MAM','JJA','SON']
    # define coordinates
    coords = {'season': (['season'], seasons),
            'lat': (['lat'], lats),
    'lon': (['lon'], lons),
             }
    # define global attributes
    attrs = {'made in':'windAnalyis/wspdComponents/FigMAP-HOV.ipynb',
    'desc': ''
    }
    ds = xr.Dataset(data_vars=data_vars,
    coords=coords,
    attrs=attrs)
    ds.to_netcdf(savenam)

    w2 = time.time()
    print(w2-w)


    w = time.time()

    tdats = [ERA_FY, ERA_DJF, ERA_MAM,ERA_JJA, ERA_SON]

    mk_slope = np.zeros([5,60,360])
    mk_p = np.zeros([5,60,360])

    for sc in range(0,5):
        testdat = tdats[sc]
        print(sc)
        for i in range(0,60):
            print(i)
            for j in range(0,360):
                tda = testdat[:,i,j]
                trend, h, p, z, Tau, s, var_s, slope, intercept = mk.original_test(tda)
                mk_slope[sc,i,j] = slope
                mk_p[sc,i,j] = p

    savenam = '/gpfs/data/greenocean/software/products/windsFromComponents/MannKendall/ERA_MK-1940-2020.nc'
    data_vars = {'mk_slope':(['season', 'lat', 'lon'], mk_slope,
    {'long_name':'MK, 1940-2020'}),
                 'mk_p':(['season', 'lat', 'lon'], mk_p,
    {'long_name':'MK, 1940-2020'}),
    }

    lats = ylist_ERA.lat.isel(lat = slice(0,60)).values
    lons = ylist_ERA.lon.values
    seasons = ['FY','DJF','MAM','JJA','SON']
    # define coordinates
    coords = {'season': (['season'], seasons),
            'lat': (['lat'], lats),
    'lon': (['lon'], lons),
             }
    # define global attributes
    attrs = {'made in':'windAnalyis/wspdComponents/FigMAP-HOV.ipynb',
    'desc': ''
    }
    ds = xr.Dataset(data_vars=data_vars,
    coords=coords,
    attrs=attrs)
    ds.to_netcdf(savenam)

    w2 = time.time()
    print(w2-w)
